In [114]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
import itertools as it
import math
import sys
sys.path.append('../')
from agents.agent import Agent
from agents.utils.utility import sigmoid_update, linear_update


### Idea
If I have a good estimate of your prior and of your error, can I learn your internal model M across time steps?
<br/>
Let's start by assuming I have a perfect estimate of your priors and error

In [115]:
# agent
a = Agent(state_size=1, model_var=0, prediction="sigmoid", behavior="sigmoid", attention="static", seed=7)
# error the agent is being fed
time = 5
#noise = np.random.uniform(0,1,50)
#print(noise)
#e_in = np.linspace(1, 0.1, time)
#e_in = e_in * noise
#print(e_in)
print(a.get_behav_model())


[0.97798951]


In [116]:
behaviors = np.random.choice([0, 1], size=(time,), p=[1./3, 2./3])
print("behaviors: ", behaviors)

behaviors:  [1 0 0 1 1]


In [117]:
# for each time step, send a behavior, get a prior, get error
errors = []
priors = []
m_hat = 0.5
for t in range(time):
    bp = a.get_behav_priors()
    priors.append(bp[0])
    a.get_world([behaviors[t]])
    dif, avg_abs_error = a.behavior_prediction_error()
    errors.append(dif[0])
    a.learn_conform()
    a.learn_predict_world()
print("errors: ", errors[:4])
print("priors: ", priors[:4])


errors:  [0.614421736848592, -0.5370530390093043, -0.40406191104371686, 0.6883958886461468]
priors:  [0.8442932825298948, 0.9081642599612204, 0.8539834046210683, 0.7975454113975015]


In [118]:
def update_weights(m, X, Y, learning_rate):
    m_deriv = 0
    #b_deriv = 0
    N = len(X)
    for i in range(N-1):
        # Calculate partial derivatives
        # -2x(y - (mx + b))
        #m_deriv += -2*X[i] * (Y[i+1] - (m*X[i]+Y[i]))
        e = X[i]*m
        m_deriv -= ( (2*m*(1-Y[i])*np.exp(-e)) * (Y[i+1]-(1 / ((((1-Y[i])*np.exp(-e))/Y[i]) + 1))) / (Y[i]*( ((((1-Y[i])*np.exp(-e))/Y[i]) + 1) ** 2)) )
        #m_deriv += sigmoid_update(prev=Y[i], error=[e])

        # -2(y - (mx + b))
        #b_deriv += -2*(Y[i] - (m*X[i] + b))

    # We subtract because the derivatives point in direction of steepest ascent
    m -= (m_deriv / float(N-1)) * learning_rate
    #b -= (b_deriv / float(N)) * learning_rate

    return m

In [119]:
m_hat = update_weights(m_hat, errors, priors, .5)
m = a.get_behav_model()
print("m_hat: ", m_hat)
#print("b: ", b)
print("m: ", m)

m_hat:  0.5000719188695288
m:  [0.97798951]
